<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.3 
# *Mining Social Media on Reddit*

## The Reddit API and the PRAW Package

The Reddit API is rich and complex, with many endpoints (https://www.reddit.com/dev/api/). It includes methods for navigating its collections, which include various kinds of media as well as comments. Fortunately, the Python library PRAW reduces much of this complexity.

Reddit requires developers to create and authenticate an app before they can use the API, but the process is much less onerus than some, and does not have waiting period for approval of new developers (as of 18 August 2018).

### 1. Create a Reddit App

Go to https://www.reddit.com/prefs/apps and click "create an app".

Enter the following in the form:

- a name for your app
- select "script" radio button
- a description
- a redirect URI

(Nb. For pulling data into a data science experiment, a local port can be used for the Redirect URI; try http://127.0.0.1:1410)


- click "create app"
- from the form that displays, copy the following to a local text file (or to this notebook):

  - name (the name you gave to your app)
  - redirect URI
  - personal use script (this is your OAuth 2 Client ID)
  - secret (this is your OAuth 2 Secret)

### 2. Register for API Access

- follow the link at https://www.reddit.com/wiki/api and read the terms of use for Reddit API access 
- fill in the form fields at the bottom 
  - make sure to enter your new OAuth Client ID where indicated
  - your use case could be something like "Training in API usage for data science projects"
  - your platform could be something like "Jupyter Notebooks / Python"
  
- click "SUBMIT"
 
- when asked for User-Agent, enter something that fits this pattern:
  `your_os-python:your_reddit_appname:v1.0 (by /u/your_reddit_username)`

### 3. Load Python Libraries

In [20]:
import praw
import requests
import json
import pprint
from datetime import datetime, date, time

### 4. Authenticate from your Python script

You could assign your authentication details explicitly, as follows:

In [21]:
my_user_agent = 'your_os-python:your_reddit_appname:v1.0 (by /u/Environmental_Dig526)'   # your user Agent string goes in here
my_client_id = 'pZZrZ_Rqlvuv8Q'   # your Client ID string goes in here
my_client_secret = 'gUNgOlg7nhMMy6LqGh3SNGTL_sRSBA'   # your Secret string goes in here

A better way would be to store these details externally, so they are not displayed in the notebook:

- create a file called "auth_reddit.json" in your "notebooks" directory, and save your credentials there in JSON format:

`{   "my_client_id": "your Client ID string goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;` "my_client_secret": "your Secret string goes in here",` <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;`"my_user_agent": "your user Agent string goes in here"` <br>
`}`

Use the following code to load the credentials:  

In [22]:
pwd()  # make sure your working directory is where the file is

'C:\\Users\\zhiwe\\OneDrive\\IOD\\Mod3 Lab'

In [23]:
path_auth = 'auth/auth_reddit.json'
auth = json.loads(open(path_auth).read())
pp = pprint.PrettyPrinter(indent=4)


my_user_agent = auth['my_user_agent']
my_client_id = auth['my_client_id']
my_client_secret = auth['my_client_secret']

In [24]:
#For debugging only:
pp.pprint(auth)

{   'my_client_id': 'pZZrZ_Rqlvuv8Q',
    'my_client_secret': 'gUNgOlg7nhMMy6LqGh3SNGTL_sRSBA',
    'my_user_agent': 'your_os-python:your_reddit_appname:v1.0 (by '
                     '/u/Environmental_Dig526)'}


Security considerations: 
- this method only keeps your credentials invisible as long as nobody else gets access to this notebook file 
- if you wanted another user to have access to the executable notebook without divulging your credentials you should set up an OAuth 2.0 workflow to let them obtain and apply their own API tokens when using your app
- if you just want to share your analyses, you could use a separate script (which you don't share) to fetch the data and save it locally, then use a second notebook (with no API access) to load and analyse the locally stored data

### 5. Exploring the API

Here is how to connect to Reddit with read-only access:

In [25]:
reddit = praw.Reddit(client_id = my_client_id, 
                     client_secret = my_client_secret, 
                     user_agent = my_user_agent)

print('Read-only = ' + str(reddit.read_only))  # Output: True

Read-only = True


In the next cell, put the cursor after the '.' and hit the [tab] key to see the available members and methods in the response object:

In [26]:
#reddit.

Consult the PRAW and Reddit API documentation. Print a few of the response members below:

In [27]:
ids = ["3rgnbke2rai6hen7ciytwcxadi", "sw7bubeycai6hey4ciytwamw3a", "t8jnufucss07"]
for thread in reddit.live.info(ids):
    print(thread.title)

/r/UkrainianConflict Live Thread
Twitch Plays Pokemon
NBA Summer League


Content in Reddit is grouped by topics called "subreddits". Content, called "submissions", is fetched by calling the `subreddit` method of the connection object (which is our `reddit` variable) with an argument that matches an actual topic. 

We also need to append a further method call to a "subinstance", such as one of the following:

- controversial
- gilded
- hot
- new
- rising
- top

One of the submission objects members is `title`. Fetch and print 10 submission titles from the 'learnpython' subreddit using one of the subinstances above:

In [28]:
for submission in reddit.subreddit('learnpython').hot(limit=10):
    print(submission.title)

Ask Anything Monday - Weekly Thread
Connecting Python and Google Sheets
LoRa import
Is it possible to write a one liner to print prime numbers in range(2,101)
Recommend a Tkinter learning resource?
[OOP Beginner] Why do I have to pass "self" to both class methods if I am using one method inside another?
Syntax error that I cant figure out
So I've never felt any dumber then I do right now
First freelancing project.
As a beginner


Now retrieve 10 authors:

In [29]:
for submission in reddit.subreddit('learnpython').hot(limit=10):
    print(submission.author)

AutoModerator
lukeflour
Katten15
kbisen
rocketjump65
anyfactor
Zypperman
LostInTheAether304
Additional_Tower
Sourih


Note that we obtained the titles and authors from separate API calls. Can we expect these to correspond to the same submissions? If not, how could we gurantee that they do?

In [30]:
for submission in reddit.subreddit('learnpython').hot(limit=10):
    print(submission.author,'-' , submission.title)

AutoModerator - Ask Anything Monday - Weekly Thread
lukeflour - Connecting Python and Google Sheets
Katten15 - LoRa import
kbisen - Is it possible to write a one liner to print prime numbers in range(2,101)
rocketjump65 - Recommend a Tkinter learning resource?
anyfactor - [OOP Beginner] Why do I have to pass "self" to both class methods if I am using one method inside another?
Zypperman - Syntax error that I cant figure out
LostInTheAether304 - So I've never felt any dumber then I do right now
Additional_Tower - First freelancing project.
Sourih - As a beginner


Why doesn't the next cell produce output?

In [31]:
for submission in submissions:
    print(submission.comments)

NameError: name 'submissions' is not defined

Answer: submissions is not defined yet

Print two comments associated with each of these submissions:

In [32]:
submissions = reddit.subreddit('learnpython').hot(limit=10)
for submission in submissions:
    top_level_comments = list(submission.comments)
    all_comments = submission.comments.list()[:2]
    for comment in all_comments:
        print(comment.body)

I am always confused about using () and [] . Can someone please help me understanding when to use them
I think I have been stuck in beginner phase (conditional statements, function etc) for so long. What should I learn next for let's say intermediate topics.
I learnt myself by using much from here. [https://developers.google.com/sheets/api](https://developers.google.com/sheets/api)

Connected it with Google Cloud Functions for small stuff reacting on events such as API calls.

Edit: A simple  google on "google sheets api cloud functions" gave me this link which I've used before: https://codelabs.developers.google.com/codelabs/cloud-function2sheet#0
Check out gspread and gspread-dataframe
Sure.

    print([2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97])
Yeah. You could do it with a list comprehension.
Not sure how much depth you want, but as far as tutorials go, [this one](https://realpython.com/python-gui-tkinter/) covers a goodly amount

Referring to the API documentation, explore the submissions object and print some interesting data:

#### Posting to Reddit

To be able to post to your Reddit account (i.e. contribute submissions), you need to connect to the API with read/write privilege. This requires an *authorised instance*, which is obtained by including your Reddit user name and password in the connection request: 

In [33]:
reddit = praw.Reddit(client_id='my client id',
                     client_secret='my client secret',
                     user_agent='my user agent',
                     username='my username',
                     password='my password')
print(reddit.read_only)  # Output: False

False


You could hide these last two credentials by adding them to your JSON file and then reading all five values at once.

In [34]:
path_auth = 'auth/auth_reddit.json'
auth = json.loads(open(path_auth).read())

In [44]:
auth.keys()

dict_keys(['my_user_agent', 'my_client_id', 'my_client_secret'])

In [45]:
reddit = praw.Reddit(client_id=auth['my_client_id'],
                     client_secret=auth['my_client_secret'],
                     user_agent=auth['my_user_agent'],
                     username='my username',
                     password='my password')
print(reddit.read_only)  # Output: False

True


>
>


>
>




---



---



> > > > > > > > > © 2021 Institute of Data


---



---



